### AB Model from spec

Package installation

In [1]:
# !pip3 install seaborn
# !pip3 install python-statemachine
# !pip3 install mesa
# !pip3 install transitions
# !pip3 install scipy
# !pip3 install cufflinks
# !pip3 install graphviz pygraphviz
# !pip3 install graphviz
# !pip3 install transitions[diagrams]
# !pip3 uninstall mesa

Package importation

In [2]:
# imports
import os
import seaborn as sns
from random import choice
import warnings
warnings.simplefilter("ignore")
import pandas as pd
import numpy as np
import mesa
from mesa import Agent, Model
from mesa.time import RandomActivation, RandomActivationByType, SimultaneousActivation
from mesa.datacollection import DataCollector
from matplotlib import pyplot as plt, patches
import scipy.stats as ss
import cufflinks as cf
cf.go_offline()
from plotly.offline import iplot
from transitions import Machine
import random
from transitions.extensions import GraphMachine
import graphviz
import timeit
from datetime import datetime
import logging
# os.environ["PATH"] += os.pathsep + '/Users/ia329/homebrew/bin' # for graphviz

Model component importation

In [3]:
import model_config as cfg
from EV.agent import EV, ChargeStation
import EV.model as model
from EV.statemachine import EVSM, LSM
from EV.model import get_evs_charge, get_evs_charge_level, get_evs_active, get_evs_queue, get_evs_travel, get_evs_not_idle, get_active_chargestations, get_eod_evs_socs, get_evs_destinations, get_ev_distance_covered
# old:  get_evs_charging,

State diagram

### Model

Model parameters

In [8]:
# 365 days = 8760 hours
# 30 days = 720 hours
# 2 days = 48 hours
# 7 days = 168 hours
# ticks =  48 # 3600 ticks = 3600 seconds = 1 hour
# no_evs = 3

# no_css = 5
# no_cps = 2
# check points = [40, 80, 120, 160, 200, 240, 280]
# # no of css must be fixed
# '40':2, '80':3, '120':4, '160':2, '200':2
ticks = 720
no_evs = 10
params = {'no_css': 5, 
          'no_cps_per_cs':{
                '1':2, '2':3, '3':4, '4':2, '5':2
                },
            'default_cppcs': 3,
            }


In [5]:
# model_run = model.EVModel(ticks=ticks, no_evs=no_evs, no_css=no_css, no_cps=no_cps)
# for i in range(ticks):
#     model_run.step()
model_run = model.EVModel(ticks=ticks, no_evs=no_evs, params=params)
for i in range(ticks):
    model_run.step()

Creating agents...

CS info: ID: 0, initialized. Charge rate: 7.5 kW.
It has 3 charging points.

CS info: ID: 1, initialized. Charge rate: 7.5 kW.
It has 3 charging points.

CS info: ID: 2, initialized. Charge rate: 7.5 kW.
It has 3 charging points.

CS info: ID: 3, initialized. Charge rate: 7.5 kW.
It has 3 charging points.

CS info: ID: 4, initialized. Charge rate: 7.5 kW.
It has 3 charging points.

EV info: ID: 5, destination name: work, journey type: Urban, max_battery: 59, energy consumption rate: 0.1643396004777234, speed: 10, State: Idle.
EV info (Cont'd): Start time: 11, distance goal: 50, soc usage threshold: 29.5, range anxiety 0.5, location: City_D.

EV info: ID: 6, destination name: work, journey type: Urban, max_battery: 41, energy consumption rate: 0.1466828317461571, speed: 10, State: Idle.
EV info (Cont'd): Start time: 14, distance goal: 50, soc usage threshold: 20.5, range anxiety 0.5, location: City_D.

EV info: ID: 7, destination name: work, journey type: Urban, max_

In [6]:
# import numpy as np

# def matrix_to_list(matrix):
#     # Convert the matrix to a nested list
#     nested_list = matrix.tolist()

#     # Return the nested list
#     return nested_list
# matrix = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
# nested_list = matrix_to_list(matrix)
# print(nested_list)


Run Model

In [7]:
# model_run.schedule.agents
run_stats = model_run.datacollector.get_model_vars_dataframe()
print(run_stats)

     EVs Charging  EVs Activated  EVs Travelling  EVs Queued  EVs Dead  \
0               0             10               0           0         0   
1               0             10               0           0         0   
2               0             10               0           0         0   
3               0             10               0           0         0   
4               0             10               0           0         0   
..            ...            ...             ...         ...       ...   
163             0             10               1           7         0   
164             0             10               1           7         0   
165             0             10               1           7         0   
166             0             10               0           7         0   
167             0             10               0           7         0   

                                      EVs Charge Level  \
0             [59, 41, 68, 46, 60, 69, 55, 45, 50, 69

Export results to CSV file

In [ ]:
# today's date as string
date_str = str(datetime.today())
# export csv
model_run.datacollector.get_model_vars_dataframe().to_csv(cfg.DATA_PATH + 'data_' + date_str[0:10] + '_' + str(no_evs) + '_EV_agent_model_output.csv')

### Data import and preprocessing

Data import and Helper functions

In [ ]:
import analysis.helper as helper
import plotly.express as px
import plotly.graph_objs as go
import pandas as pd

In [ ]:
data = pd.read_csv(cfg.DATA_PATH +'modeldatadata_2023-03-19_500_EV_agent_model_output.csv')

data = data.drop(columns=['Unnamed: 0'])

In [ ]:
# data.info()
data.head()

In [ ]:
a = helper.unpack_and_join(data, 'EVs Charge Level')
print(a)

In [ ]:
# # EV charge level per EV per timestep - 20 EVs unpacked
# newdf = a[['EVs Charge Level', 'EVs Charge Level_unpacked_0', 'EVs Charge Level_unpacked_1', 'EVs Charge Level_unpacked_2', 'EVs Charge Level_unpacked_3', 'EVs Charge Level_unpacked_4', 'EVs Charge Level_unpacked_5', 'EVs Charge Level_unpacked_6', 'EVs Charge Level_unpacked_7', 'EVs Charge Level_unpacked_8', 'EVs Charge Level_unpacked_9', 'EVs Charge Level_unpacked_10', 'EVs Charge Level_unpacked_11', 'EVs Charge Level_unpacked_12', 'EVs Charge Level_unpacked_13', 'EVs Charge Level_unpacked_14', 'EVs Charge Level_unpacked_15', 'EVs Charge Level_unpacked_16', 'EVs Charge Level_unpacked_17', 'EVs Charge Level_unpacked_18', 'EVs Charge Level_unpacked_19']]
# newdf.head()

# EV charge level per EV per timestep - 20 EVs unpacked
newdf = a[['EVs Charge Level', 'EVs Charge Level_unpacked_0', 'EVs Charge Level_unpacked_1', 'EVs Charge Level_unpacked_2', 'EVs Charge Level_unpacked_3', 'EVs Charge Level_unpacked_4']]
newdf.head()

### Batching

In [ ]:
from mesa.batchrunner import BatchRunner

EVcounts = (100,500,1000)
cpcounts = (1,2)
tickcounts = (24,48)
model_reporters={'EVs Charged': get_evs_charged,
                'EVs Activated': get_evs_active,
                'EVs Travelling': get_evs_travel,
                'EVs Charge Level': get_evs_charge_level,
                'EVs Currently charging': get_evs_charging,
                'EVs Not Idle': get_evs_not_idle,
                'EOD Battery SOC': get_eod_evs_socs,
                'EVs Destinations': get_evs_destinations,
                }
# parameters = {"no_evs": range(1000,20000,3000), "no_cps": 1}
parameters = {"no_evs": EVcounts, "no_cps": cpcounts, "ticks": tickcounts}
batch_run = BatchRunner(model.EVModel, parameters, max_steps=24, iterations=1, model_reporters= model_reporters) #iterations=1
batch_run.run_all()

In [ ]:
# batch_df = batch_run.get_model_vars_dataframe()

In [ ]:
# print(batch_df)

### Visualisations

Scatter plot

In [ ]:
# plot_data_lines(newdf, 'Timestep', ['EVs Charge Level_unpacked_0', 'EVs Charge Level_unpacked_1', 'EVs Charge Level_unpacked_2', 'EVs Charge Level_unpacked_3', 'EVs Charge Level_unpacked_4'])

### Scrapbook